Assignment 2

In [1]:
import numpy as np

def strassen(A , B):
    n = len(A)
    
    if n <= 2:
        return np.dot(A,B) #จับ maxtrix มา dot กันได้ออกมาเป็นค่า scalar
    mid = n // 2  #หาจุดกึ่งกลางของ maxtrix 
    A11 = A[:mid , :mid] #ตำเเหน่ง 0:2 2:4 บนซ้ายจะได้ (0,0)(0,1),(1,0)(1,1)
    A12 = A[:mid , mid:] #ตำเเหน่ง 0:2	2:4 บนขวาจะได้ (0,2)(0,3)(1,2)(1,3)
    A21 = A[:mid , :mid] #ตำเเหน่ง 2:4	0:2 ล่างซ้ายจะได้ (2,0)(2,1)(3,0)(3,1)
    A22 = A[:mid , mid:] #ตำเเหน่ง 2:4	2:4 ล่างขวาจะได้ (2,2)(2,3)(3,2)(3,3)
    
    B11 = B[:mid , :mid] #เหมือน A เเต่เปลี่ยนมา B
    B12 = B[:mid , mid:]
    B21 = B[:mid , :mid]
    B22 = B[:mid , mid:]
    
    P1 = strassen(A11, B12 -B22) #สูตรการคูณของ strassen 
    P2 = strassen(A11 + B12 , B22)
    P3 = strassen(A21 + A22 , B11)
    P4 = strassen(A22, B21 - B11)
    P5 = strassen(A11 + A22, B11 + B22)
    P6 = strassen(A12 - A22, B21 + B22)
    P7 = strassen(A11 - A21, B11 + B12)
    
    C11 = P5 + P4 - P2 + P6 #จับค่าที่ได้มา Combination กัน
    C12 = P1 + P2
    C21 = P3 + P4
    C22 = P5 + P1 - P3 - P7
    
    C = np.vstack((np.hstack((C11 , C12)) , np.hstack((C21 , C22)))) #จับค่าเเต่ละ Quarter กลับมาที่เดิมเป็น maxtrix
    return C

In [3]:
A = np.array([[2, 3], [7, 5]])
B = np.array([[6, 8], [4, 2]])
C = strassen(A, B)
print("Matrix C (Result of A * B):\n", C)

Matrix C (Result of A * B):
 [[24 22]
 [62 66]]


กรณี maxtrix ใหญ่กว่า 2x2 ใช้วิธี padding หรือกาาเติม 0 ให้เป็น 2กำลังn เช่น ถ้าเป็น 3x3 ก็เติม 0 ให้เป็น 4x4 หรือ 2กำลังสอง

In [ ]:
import numpy as np

def reshapemaxtrix(A , B):
    original_row = A.shape[0]
    original_colum = B.shape[1]
    print( original_row , original_colum)
    
    max_dim = max(A.shape[0] , A.shape[1] , B.shape[1]) #ต้องหา max เพราะ Strassen บังคับว่าต้องเป้น maxtrix nxn เเละ n ต้องเป็นเลขคู่ที่หาร 2 ลงไปได้เรื่อยๆ
    print(max_dim)
    
    n = 2**int(np.ceil(np.log2(max_dim))) #คำนวณหาขนาด maxtrix ที่ยกกำลัง 2 ใช้ Log เพื่อหา เช่น 3x3 ก็จะเป็น log2(3) = 1.58 เเล้วใช้ ceil ปัด 1.58 เป็น 2 2*2 ก็จะได้ n = 4
    
    A_pad = np.pad(A , ((0 , n-A.shape[0]) , (0 , n-A.shape[1])) , mode="constant")
    B_pad = np.pad(B , ((0 , n-B.shape[0]) , (0 , n-B.shape[1])) , mode="constant")
    
    C_pad = strassen(A_pad , B_pad)
    
    return C_pad #[:orig_rows, :orig_cols] ใช้การ slice เพื่อเอาเเค่ 3x3 ได้
    
A_3x3 = np.array([[1, 2, 3], 
                  [4, 5, 6], 
                  [7, 8, 9]])

B_3x3 = np.array([[9, 8, 7], 
                  [6, 5, 4], 
                  [3, 2, 1]])
reshapemaxtrix(A_3x3 , B_3x3)

3 3
3


array([[ 20,  42,  64,   0],
       [134, 105,  76,   0],
       [ 48,  42,  36,   0],
       [120, 105,  90,   0]])

การหา Closet pair โดย Divide and conquer

In [14]:
import math

def distance(p1 , p2):
    return math.sqrt((p1[0] - p2[0])**2 + (p1[1] - p2[1])**2)

def stripCloset(strip , d):
    min_dist = d
    
    strip.sort(key=lambda point:point[1])
    
    for i in range(len(strip)):
        for j in range(i +1 , len(strip)):
            if (strip[j][1] - strip[i][1]) < min_dist:
                min_dist = min(min_dist , distance(strip[i] , strip[j]))
            else:
                break
    return min_dist

def minDistUtill(points , left , right):
    if right - left <= 2:
        min_dist = float("inf")
        for i in range(left , right):
            for j in range(i+1 , right):
                min_dist = min(min_dist , distance(points[i] , points[j]))
        return min_dist
    
    mid =  (left + right) // 2
    mid_x = points[mid][0]
    
    dl = minDistUtill(points , left , mid)
    dr = minDistUtill(points , mid , right)
    
    d = min(dl , dr)
    
    strip = []
    for i in range(left , right):
        if abs(points[i][0] - mid_x) < d:
            strip.append(points[i])
            
    stripDist = stripCloset(strip ,d)
    
    return min(d , stripDist)

def minDistance(points):
    n = len(points)
    
    points.sort(key=lambda point:point[0])
    
    return minDistUtill(points , 0 , n)

if __name__ == "__main__" :
    points = [[-1, -2], [0, 0], [1, 2], [2, 3]]
    res = minDistance(points)
    
    
    print(f"{res:6f}")
    


    
    

1.414214


quickhull

In [25]:
hull = set()

def findside(p1 , p2 , p):
    val = (p[1] - p1[1]) * (p2[0] - p1[0]) - (p2[1] - p1[1]) * (p[0] - p1[0])
    
    if val > 0:
        return 1
    if val < 0:
        return -1
    return 0


def lineDist(p1 , p2 , p):
    return abs((p[1] - p1[1]) * (p2[0] - p1[0]) - (p2[1] - p1[1]) * (p[0] - p1[0]))

def quickHull(a , n , p1 , p2 , side):
    ind = -1
    max_dist = 0
    
    for i in range(n):
        temp = lineDist(p1 , p2 , a[i])
        
        if (findside(p1 , p2 , a[i]) == side) and (temp > max_dist):
            ind = i
            max_dist = temp
            
    if ind == -1:
        hull.add("$".join(map(str , p1)))
        hull.add("$".join(map(str , p2)))
        return
    
    quickHull(a , n , a[ind], p1 , -findside(a[ind] , p1 , p2))
    quickHull(a , n , a[ind], p2 , -findside(a[ind] , p2 , p1))
    
def printHull(a , n):
    if (n < 3):
        print("Convex hull not possible")
        return
    
    min_x = 0
    max_x = 0
    for i in range(1 , n):
        if a[i][0] < a[min_x][0]:
            min_x = i 
        if a[i][0] > a[max_x][0]:
            max_x = i
    quickHull(a , n , a[min_x] , a[max_x] , 1)
    quickHull(a , n , a[min_x] , a[max_x] , -1)
    
    print("The possible is convex hull are:")
    
    for element in hull:
        x = element.split("$")
        print("(" , x[0] , "," , x[1] , "," , end = "")
    
    

In [26]:
a = [[0, 3], [1, 1], [2, 2], [4, 4],
     [0, 0], [1, 2], [3, 1], [3, 3]]
n = len(a)
printHull(a, n)

The possible is convex hull are:
( 4 , 4 ,( 0 , 3 ,( 3 , 1 ,( 0 , 0 ,